In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import mysql.connector

In [2]:
def extract_book_links(doc):
    """ Fetch  url links from page"""
    h3_tags = doc.find_all('h3')
    book_links = []
    base_url = "http://books.toscrape.com/catalogue/"

    for tag in h3_tags:
        book_links.append(base_url + tag.contents[0]['href'].replace('../../../',''))

    return book_links

In [3]:
def extract_table_heads(topic_doc):
    """ Extract headers of Product Information for each book """
    desc_tags = topic_doc.find_all('th')                    
    desc = []                                              
    for tag in desc_tags:
        desc.append(tag.text)
        
    return desc

In [4]:
def extract_table_data(topic_doc):
    """Extract output(value) of Product Information for each book """
    tabledata_tags = topic_doc.find_all('td')
    tbdata = []                                             
    for tag in tabledata_tags:                  
        tbdata.append(tag.text)
        
    return tbdata

In [5]:
def extract_paragraph(topic_doc):
    """ Extract product description for each book """
    div_tags = topic_doc.find('div',{'class':'content'})
    para_tags = div_tags.find_all('p')
    
    return para_tags[3].text

In [6]:
def extract_title_of_books(topic_doc):
    """ Extract title name of the books"""
    tit_tag = topic_doc.find('h1')
    return tit_tag.text

In [7]:
def csv_home_page(book_links):
    """Create dataframe containing all the above information for each book """
    list3 = []
    list1 = []
    list2 = []
    for i in book_links:                                        
        response = requests.get(i)
        topic_doc = BeautifulSoup(response.text, 'html.parser')                                           
        table = dict(zip( extract_table_heads(topic_doc), extract_table_data(topic_doc)))
        list3.append(table)   
        list1.append(extract_paragraph(topic_doc))
        list2.append(extract_title_of_books(topic_doc))
    df = pd.DataFrame(list3)
    df.pop('Product Type')
    df1 = pd.DataFrame(list1,columns =['Description'])
    df2 = pd.DataFrame(list2,columns =['Title'])
    final = pd.concat([df2,df1,df],axis=1)
    csv_data = final.to_csv('csv_data_downloaded.csv', index=False) 
    return final

In [8]:
def get_topics_page(topics_url):
    """Fetch inspect elements for each url """
    response = requests.get(topics_url)
    page_contents = response.text
    doc = BeautifulSoup(page_contents, 'html.parser')
    return doc

In [9]:
topics_urls = ['http://books.toscrape.com/catalogue/category/books/poetry_23/index.html',
              'http://books.toscrape.com/catalogue/category/books/science_22/index.html']

doc = get_topics_page(topics_urls[0])
df=csv_home_page(extract_book_links(doc))

doc1 = get_topics_page(topics_urls[1])
df1=csv_home_page(extract_book_links(doc1))

df=df.append(df1, ignore_index = True)
csv_data = df.to_csv('books_data.csv', index=False)

In [10]:
df = df.rename(columns={'Price (excl. tax)': 'Price_excl_tax', 'Price (incl. tax)': 'Price_incl_tax','Tax':'Tax','Number of reviews':'No_of_Reviews'})  


In [11]:
df[df.columns[3:6]] = df[df.columns[3:6]].replace(' ', '', regex=True)
df[df.columns[3:6]] = df[df.columns[3:6]].replace('[\Â£]', '', regex=True).astype(float)
df[df.columns[7]]=df[df.columns[7]].astype(int)

In [12]:
import mysql.connector
try:
    with open("secrets.txt") as f:
        lines = f.readlines()
        username = lines[0].strip()
        password = lines[1].strip()
        database = lines[2].strip()
    # print(username,password,database)
    connection = mysql.connector.connect(host='localhost',
                                             database=database,
                                             user=username,
                                             password=password)

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS book;')
        print('Creating table....')
        cursor.execute("CREATE TABLE book (Title VARCHAR(200) \
                        , Description TEXT  ,\
                        UPC VARCHAR(100) , Price_excl_tax FLOAT(8,2),Price_incl_tax FLOAT(8,2), Tax FLOAT(5,2), Availability VARCHAR(200), No_of_Reviews INT \
                        )")
        print("book table is created....")

        for index, row in df.iterrows():
            #print(df)
            insert_stmt = (
                          "INSERT INTO book(Title, Description, UPC, Price_excl_tax, Price_incl_tax, Tax, Availability, No_of_Reviews) "
                          "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
                            )
            data = (row.Title, row.Description, row.UPC, row.Price_excl_tax, row.Price_incl_tax, row.Tax, row.Availability, row.No_of_Reviews)
            cursor.execute(insert_stmt, data)

            #cursor.execute('INSERT INTO book(Title, \
            #      Description, UPC, Price_excl_tax, Price_incl_tax, Tax, Availability, No_of_Reviews) \
            #      VALUES(%s, %s, %s, %s, %s, %s, %s, %s)', 
            #      row)
        #close the connection to the database.
        connection.commit()
        cursor.close()
        print("Done")
except Error as e:
    print("Error while connecting to MySQL", e)

Connected to MySQL Server version  8.0.27
You're connected to database:  ('mybooks',)
Creating table....
book table is created....
Done
